Tests to run GenIce on Jupyter and Google Colaboratory.

In [2]:
%pip install genice

You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
from logging import getLogger, INFO
logger = getLogger()
logger.setLevel(INFO)

from genice import genice as gi
from genice.importer import safe_import

# optionをどう渡すのか。
lattice_maker = safe_import("lattice", "5")
formatter     = safe_import("format", "gromacs").Format()
water         = safe_import("molecule", "tip4p").Molecule()
guests        = None
lat = gi.GenIce(lattice_maker)
ice = lat.generate_ice(water, formatter=formatter)
print(ice)

('lattice', '5', {})
Options are given but the module does not accept them.
('format', 'gromacs', {})
('molecule', 'tip4p', {})
INFO:root:Cell dimension:
INFO:root:  a = 9.19977812589
INFO:root:  b = 7.52346280577
INFO:root:  c = 10.327299740014167
INFO:root:  A = 90.0
INFO:root:  B = 109.2
INFO:root:  C = 90.0
INFO:root:HB connectivity is not defined.
INFO:root:Bond length (specified): 3
INFO:root:Target Density: 1.23983176668
INFO:root:Original Density: 0.0012398317666827844
INFO:root:Bond length (scaled, nm): 0.3000000000002246
INFO:root:Stage1: Replication.
INFO:root:  Number of water molecules: 28
INFO:root:  Pairs are not given explicitly.
INFO:root:  Estimating the bonds according to the pair distances.
INFO:root:  Number of water nodes: 28
INFO:root:Cell dimension:
INFO:root:  a = 0.9199778125896888
INFO:root:  b = 0.7523462805775633
INFO:root:  c = 1.0327299740021898
INFO:root:  A = 90.0
INFO:root:  B = 109.2
INFO:root:  C = 90.0
INFO:root:Stage1: end.
INFO:root:Stage2: Graph 

Generated by GenIce https://github.com/vitroid/GenIce 
112
    1ICE     OW    1   0.658   0.105   0.469
    1ICE    HW1    2   0.671   0.176   0.532
    1ICE    HW2    3   0.725   0.041   0.491
    1ICE     MW    4   0.668   0.106   0.480
    2ICE     OW    5   0.373   0.103   0.499
    2ICE    HW1    6   0.370   0.156   0.578
    2ICE    HW2    7   0.466   0.095   0.478
    2ICE     MW    8   0.384   0.109   0.506
    3ICE     OW    9  -0.083   0.653   0.501
    3ICE    HW1   10  -0.071   0.609   0.585
    3ICE    HW2   11  -0.084   0.582   0.437
    3ICE     MW   12  -0.081   0.638   0.503
    4ICE     OW   13   0.205   0.641   0.472
    4ICE    HW1   14   0.112   0.660   0.485
    4ICE    HW2   15   0.249   0.725   0.486
    4ICE     MW   16   0.199   0.654   0.475
    5ICE     OW   17   0.498   0.479   0.964
    5ICE    HW1   18   0.500   0.532   0.884
    5ICE    HW2   19   0.405   0.472   0.985
    5ICE     MW   20   0.486   0.485   0.957
    6ICE     OW   21   0.547   0.475   0.

In [7]:
%tb

SystemExit: 1

理想的なAPIはどんな感じだろう。

```python
from genice.formats import ice3 
lattice_maker = ice3(options)
```
この書法だと、オプションはどうやって渡すのか?

書き方はシンプルだが、動的に氷の名前を選べるようにするのが却って難しくなる。
さらに、ユーザー定義のモジュールを呼びにくくなる。

一方、現状の仕様の最大の問題は、moduleだと、一度読み込まれたあと、kernelをリスタートしない限り読まれっぱなしだということ。これは致命的に問題。Jupyterの用法にまったくそぐわない。

この意味で、moduleそのものを利用するのではなく、class定義をmoduleから読みこむようにすべき。インスタンスはその都度生成する。

https://www.geeksforgeeks.org/how-to-dynamically-load-modules-or-classes-in-python/
を参考に、
* ユーザーが書いたmoduleファイルの中でクラスを定義する。
* それを必要な時に読みこんでinstanceを生成する。
* できるだけ、safe_importの枠組みは維持する。
* moduleはクラスではないのか? ちがう。moduleをclassのようには使えない。classが欲しいならclass定義するしかない。Sigh.

```python
lattice_class = safe_import("lattice", "5", "Ice5") # type, module name, class name
```

* 同じ名前のクラスを定義すると、上書きされる? いや、そんなことはないはず。
* `genice.lattices.ice5.Lattice`と`genice.lattices.ice3.Lattice`はクラス名が同じでも区別可能。

```python
lattice_module = safe_import("lattice", "5") # type, module name
lattice        = lattice_module.Lattice(options) # make an instance
```
* `.Lattice`の部分をモジュール依存の名前にしてしまうと、呼び出しが面倒になるだけでメリットがない。
* moduleにはLatticeクラスが1つだけ定義されているとする。そして、そのテンプレートを作り、継承する。
* この方針なら、safe_importを変更する必要はない。むしろ、オプションの解釈をクラスにまかせられるので、もっとシンプルにできる。

* もっとオブジェクト指向っぽいワークフローにしたい。
* 何が何に依存していて、何が何を内包すべきかをもう一度ちゃんと考える。
* `lattice`は基本的には単位胞のなかの酸素の位置の情報のみを持つ。水素などの情報は、formatが決まるまでは必要ない可能性があるから。
* latticeにformatterの情報を与えるというよりは、formatterにlatticeの情報や何やかやを食わせるのが正しい気がする。

```python
lattice_module = safe_import("lattice", "5") # type, module name
lattice        = lattice_module.Lattice(options) # make an instance
formatter      = safe_import("format", "python").Format(lattice, water, guests)
```

* その場合、じゃあ、analiceはどういう動作になる?

```python
loader_module  = safe_import("loader", "gro") # type, module name
lattices_iter  = loader_module.Loader(filename_pattern, options) # make an instance
formatter      = safe_import("format", "python").Format(lattices_iter, water, guests)
formatter.dump("filename")
```

* Format() コンストラクタがパラメータをうけとり、dumpの中で分子配置を生成する。
* dumpはすべての構造を読みこんでから処理してもいいし、逐次出力してもいい。
* こうすれば、例えばGenIceで氷3と氷5をまとめて生成することもできる。

```python
lattice5 = safe_import("lattice", "5").Lattice()
lattice3 = safe_import("lattice", "3").Lattice()
formatter= safe_import("format", "python").Format([lattice3, lattice5], water, guests)
```

* でもそうすると、waterやguestsの場所がここでいいのか、すこし不安になるね。
* 将来、もしanaliceがゲスト位置も読みこむようになるなら? それでもいいか。

* ここでいうguestというのは、分子の名前とクラスの対応を示しているだけ。どの場所にするかはどこでどうやって指示する?

* 全optionが合理的に指定できるかどうかを、ひきつづき検討する。


In [3]:
# まず、formatsをclassに変換した。成功。

import sys
from logging import getLogger, INFO, DEBUG
logger = getLogger()
logger.setLevel(INFO)

from genice import genice as gi
from genice.importer import safe_import

# まず、formatterだけ新書式にしてみよう。
lattice_maker = safe_import("lattice", "5") 
formatter     = safe_import("format", "python").Format()
water         = safe_import("molecule", "4site")
guests        = None
# 以下の構造はさいごに変更する。
lat = gi.GenIce(lattice_maker)
ice = lat.generate_ice(water, formatter=formatter)
print(ice)

INFO:root:  
INFO:root:  [1] M. W. Mahoney and W. L. Jorgensen, A five-site model for liquid water and the reproduction of the density anomaly by rigid, nonpolarizable potential functions, J. Chem. Phys. 112 (2000) 8910-8922. [Back]
INFO:root:  [2] W. L. Jorgensen, J. Chandrasekhar, J. D. Madura, R. W. Impey, and M. L. Klein, Comparison of simple potential functions for simulating liquid water, J. Chem. Phys. 79 (1983) 926-935
INFO:root:  [3] W. L. Jorgensen and J. D. Madura, Temperature and size dependence for monte carlo simulations of TIP4P water, Mol. Phys. 56 (1985) 1381-1392.
INFO:root:No rotmatrices in lattice
INFO:root:Cell dimension:
INFO:root:  a = 9.19977812589
INFO:root:  b = 7.52346280577
INFO:root:  c = 10.327299740014167
INFO:root:  A = 90.0
INFO:root:  B = 109.2
INFO:root:  C = 90.0
INFO:root:HB connectivity is not defined.
INFO:root:Bond length (specified): 3
INFO:root:Target Density: 1.23983176668
INFO:root:Original Density: 0.0012398317666827844
INFO:root:Bond length

"""

Reshaping the unit cell.
  i:[1 0 0]
  j:[0 1 0]
  k:[0 0 1]
"""
bondlen=3.0000000000022458
coord='relative'

import numpy as np
cell=np.array([[9.19977813, 0.00000000, 0.00000000], [0.00000000, 7.52346281, 0.00000000], [-3.39630444, 0.00000000, 9.75285784], ])
density=1.23983176668
waters="""
    0.8993    0.1404    0.4854
    0.6007    0.1404    0.5146
    0.1007    0.8596    0.5146
    0.3993    0.8596    0.4854
    0.8993    0.6404    0.9854
    0.6007    0.6404    0.0146
    0.1007    0.3596    0.0146
    0.3993    0.3596    0.9854
    0.7751    0.8475    0.7477
    0.7249    0.8475    0.2523
    0.2249    0.1525    0.2523
    0.2751    0.1525    0.7477
    0.7751    0.3475    0.2477
    0.7249    0.3475    0.7523
    0.2249    0.6525    0.7523
    0.2751    0.6525    0.2477
    0.9629    0.4435    0.6544
    0.5371    0.4435    0.3456
    0.0371    0.5565    0.3456
    0.4629    0.5565    0.6544
    0.9629    0.9435    0.1544
    0.5371    0.9435    0.8456
    0.0371    0.05